## Batch processing with Argo Worfklows

In this notebook we will dive into how you can run batch processing with Argo Workflows and Seldon Core.

Dependencies:

* Seldon core installed as per the docs with an ingress
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Seldon Batch Processor Python Module
The Seldon Core Python module contains a batch process optimisation feature which allows us to send data in batches for processing with extensible throughput.

In [ ]:
### Creating a Seldon Model

In [126]:
%%bash
kubectl apply -f - << END
    apiVersion: machinelearning.seldon.io/v1alpha2
    kind: SeldonDeployment
    metadata:
      name: sklearn-model
    spec:
      predictors:
      - graph:
          children: []
          implementation: SKLEARN_SERVER
          modelUri: gs://seldon-models/sklearn/iris
          name: classifier
        name: default
        replicas: 1
END

seldondeployment.machinelearning.seldon.io/sklearn-model created


In [147]:
!mkdir -p assets/
!rm assets/output-data.txt || echo "Output file already removed"

In [140]:
with open("assets/input-data.txt", "w") as f:
    for i in range(10000):
        f.write('{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "' + str(i) + '"}}\n')

In [148]:
%%bash

seldon-batch-processor \
    --deployment-name "sklearn-model" \
    --namespace "default" \
    --host "localhost:80" \
    --workers "100" \
    --retries "3" \
    --input-data-path "assets/input-data.txt" \
    --output-data-path "assets/output-data.txt"

echo "First 10 outputs collected out of 10,000:"
head -10 "assets/output-data.txt"

First 10 outputs collected out of 10,000:
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"16"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"0"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"11"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"24"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"98"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"99"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"88"}}
{"data"

2020-05-18 22:20:46.777863: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-05-18 22:20:46.778538: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-05-18 22:20:46.778741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


## Seldon Core Batch with Object Store

In some cases we may want to read the data from an object source.

In this case we will show how you can read from an object store, in this case minio.

The workflow will look as follows:

![](assets/seldon-batch.jpg)

For this we will assume you have installed the Minio (mc) CLI - we will use a Minio client in the cluster but you can use another object store provider like S3, Google Cloud, Azure, etc.

### Set up Minio in your cluster

In [481]:
%%bash 
helm install minio stable/minio \
    --set accessKey=minioadmin \
    --set secretKey=minioadmin \
    --set image.tag=RELEASE.2020-04-15T19-42-18Z

NAME: minio
LAST DEPLOYED: Thu Apr 30 10:57:00 2020
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Minio can be accessed via port 9000 on the following DNS name from within your cluster:
minio.default.svc.cluster.local

To access Minio from localhost, run the below commands:

  1. export POD_NAME=$(kubectl get pods --namespace default -l "release=minio" -o jsonpath="{.items[0].metadata.name}")

  2. kubectl port-forward $POD_NAME 9000 --namespace default

Read more about port forwarding here: http://kubernetes.io/docs/user-guide/kubectl/kubectl_port-forward/

You can now access Minio server on http://localhost:9000. Follow the below steps to connect to Minio server with mc client:

  1. Download the Minio mc client - https://docs.minio.io/docs/minio-client-quickstart-guide

  2. mc config host add minio-local http://localhost:9000 minioadmin minioadmin S3v4

  3. mc ls minio-local

Alternately, you can use your browser or the Minio SDK to access the server - ht

### Forward the Minio port so you can access it

You can do this by runnning the following command in your terminal:
```
kubectl port-forward svc/minio 9000:9000
    ```
    
### Configure local minio client

In [483]:
!mc config host add minio-local http://localhost:9000 minioadmin minioadmin

Added `minio-local` successfully.


### Configure bucket for Argo Artefact passing

In [76]:
%%writefile assets/argo-config.yaml
---
apiVersion: v1
kind: ConfigMap
metadata:
  name: workflow-controller-configmap
data:
  config: |
    artifactRepository:
      s3:
        bucket: argo-artifacts
        endpoint: minio.default.svc.cluster.local:9000
        insecure: true
        accessKeySecret:
          name: minio
          key: accesskey
        secretKeySecret:
          name: minio
          key: secretkey

Overwriting assets/argo-config.yaml


### Make sure the configmap is in the same location as your argo controller

In [108]:
!kubectl apply -n argo -f assets/argo-config.yaml

configmap/workflow-controller-configmap unchanged


### Create the bucket for artifact passing

In [572]:
!mc mb minio-local/argo-artifacts

Bucket created successfully `minio-local/argo-artifacts`.


### Create some input for our model

We will create a file that will contain the inputs that will be sent to our model

In [484]:
with open("assets/input-data.txt", "w") as f:
    for i in range(10000):
        f.write('{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "' + str(i) + '"}}\n')

### Check the contents of the file

In [485]:
!wc -l assets/input-data.txt
!head assets/input-data.txt

10000 assets/input-data.txt
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "0"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "1"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "2"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "3"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "4"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "5"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "6"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "7"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "8"}}
{"data":{"ndarray":[[1, 2, 3, 4]]}, "meta": { "puid": "9"}}


### Upload the file to our minio

In [486]:
!mc mb minio-local/data
!mc cp assets/input-data.txt minio-local/data/

Bucket created successfully `minio-local/data`.
...-data.txt:  614.15 KiB / 614.15 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 9.74 MiB/s 0s

### Create Argo Workflow

In order to create our argo workflow we have made it simple so you can leverage the power of the helm charts.

Before we dive into the contents of the full helm chart, let's first give it a try with some of the settings.

We will run a batch job that will set up a Seldon Deployment with 10 replicas and 100 batch client workers to send requests.

In [8]:
!ls helm-charts/

seldon-batch-workflow


In [119]:
!helm template seldon-batch-workflow helm-charts/seldon-batch-workflow \
    --set workflow.name=seldon-batch-process \
    --set seldonDeployment.create=false \
    --set seldonDeployment.name="seldon-batch" \
    --set seldonDeployment.replicas=10 \
    --set batchWorker.workers=100 \
    --set minio.inputDataPath="s3://data/input-data.txt" \
    --set minio.outputDataPath="s3://data/output-data-{{workflow.uid}}.txt" \
    | argo submit -

Name:                seldon-batch-process
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Mon May 18 21:55:31 +0100 (now)


In [120]:
!argo list

NAME                   STATUS    AGE   DURATION   PRIORITY
seldon-batch-process   Running   46s   46s        0


In [121]:
!argo get seldon-batch-process

Name:                seldon-batch-process
Namespace:           default
ServiceAccount:      default
Status:              Running
Created:             Mon May 18 21:55:31 +0100 (53 seconds ago)
Started:             Mon May 18 21:55:31 +0100 (53 seconds ago)
Duration:            53 seconds

STEP                                                             PODNAME                          DURATION  MESSAGE
 ● seldon-batch-process (seldon-batch-process)                                                              
 ├---✔ create-seldon-resource (create-seldon-resource-template)  seldon-batch-process-3626514072  1s        
 ├---✔ wait-seldon-resource (wait-seldon-resource-template)      seldon-batch-process-2052519094  30s       
 └---● process-batch-inputs (process-batch-inputs-template)      seldon-batch-process-50851621    20s       


In [112]:
!argo logs -w seldon-batch-process 

create-seldon-resource:	time="2020-05-18T20:52:21Z" level=info msg="Starting Workflow Executor" version=vv2.7.4+50b209c.dirty
create-seldon-resource:	time="2020-05-18T20:52:21Z" level=info msg="Creating a docker executor"
create-seldon-resource:	time="2020-05-18T20:52:21Z" level=info msg="Executor (version: vv2.7.4+50b209c.dirty, build_date: 2020-04-16T16:37:57Z) initialized (pod: default/seldon-batch-process-3626514072) with template:\n{\"name\":\"create-seldon-resource-template\",\"arguments\":{},\"inputs\":{},\"outputs\":{},\"metadata\":{},\"resource\":{\"action\":\"create\",\"manifest\":\"apiVersion: machinelearning.seldon.io/v1\\nkind: SeldonDeployment\\nmetadata:\\n  name: \\\"seldon-844299e9-abf7-4742-9d4d-eeb405433569\\\"\\n  namespace: default\\n  ownerReferences:\\n  - apiVersion: argoproj.io/v1alpha1\\n    blockOwnerDeletion: true\\n    kind: Workflow\\n    name: \\\"seldon-batch-process\\\"\\n    uid: \\\"844299e9-abf7-4742-9d4d-eeb405433569\\\"\\nspec:\\n  name: \\\"seldon

## Check output in object store

We can now visualise the output that we obtained in the object store.

First we can check that the file is present:

In [114]:
import json
wf_arr = !argo get seldon-batch-process -o json
wf = json.loads("".join(wf_arr))
WF_ID = wf["metadata"]["uid"]
print(f"Workflow ID is {WF_ID}")

Workflow ID is 844299e9-abf7-4742-9d4d-eeb405433569


In [115]:
!mc ls minio-local/data/output-data-"$WF_ID".txt

[2020-05-18 21:53:16 BST]  1.3MiB output-data-844299e9-abf7-4742-9d4d-eeb405433569.txt


Now we can output the contents of the file created using the `mc head` command.

In [116]:
!mc cp minio-local/data/output-data-"$WF_ID".txt assets/output-data.txt
!head assets/output-data.txt

...33569.txt:  1.31 MiB / 1.31 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 26.12 MiB/s 0s{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"4"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"0"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"3"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"1"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"100"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.9956334415074478]]},"meta":{"puid":"6"}}
{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.0006985194531162841,0.003668039039435755,0.99563344150744

In [122]:
!argo delete seldon-batch-process

Workflow 'seldon-batch-process' deleted
